In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import datetime
import calendar
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('/home/najeeb/Desktop/Dataset/Shelter Animals/train.csv',index_col=0)
data_test = pd.read_csv('/home/najeeb/Desktop/Dataset/Shelter Animals/test.csv',index_col=0)

In [3]:
data.describe()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,6374,22918,5,16,2,5,44,1380,366
top,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,136,19,10769,7816,15595,9779,3969,8810,2824


In [4]:
data_test.describe()

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,8231,11456,11456,11456,11450,11456,11456
unique,3712,10575,2,5,45,913,277
top,Bella,2014-10-20 09:00:00,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,60,8,6656,4235,1768,3777,1219


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26729 entries, A671945 to A706629
Data columns (total 9 columns):
Name              19038 non-null object
DateTime          26729 non-null object
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
dtypes: object(9)
memory usage: 2.0+ MB


In [6]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11456 entries, 1 to 11456
Data columns (total 7 columns):
Name              8231 non-null object
DateTime          11456 non-null object
AnimalType        11456 non-null object
SexuponOutcome    11456 non-null object
AgeuponOutcome    11450 non-null object
Breed             11456 non-null object
Color             11456 non-null object
dtypes: object(7)
memory usage: 716.0+ KB


In [7]:
data.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [8]:
data_test.head()

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
ID,,,,,,,
1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White


In [9]:
data = data[pd.notnull(data['AgeuponOutcome'])]

In [10]:
data_test.loc[pd.isnull(data_test['AgeuponOutcome']),'AgeuponOutcome'] = '1 year'

In [11]:
nb_days = {'year':365,'month':30,'week':7,'day':1}

In [12]:
data['Age'] = map(lambda x:int(x.split()[0])*nb_days[x.split(' ')[1].strip('s')],data['AgeuponOutcome'])
data_test['Age'] = map(lambda x:int(x.split()[0])*nb_days[x.split(' ')[1].strip('s')],data_test['AgeuponOutcome'])

In [13]:
color = map(lambda x: x.split('/')[0].split(' ')[0],data['Color'].unique().tolist())
unique_color = set(color)
color_map = dict((i,w+1) for w,i in enumerate(unique_color))
color_map['Other'] = 30

In [14]:
data['Color_map'] = map(lambda x:color_map[x.split('/')[0].split(' ')[0]],data['Color'])
data_test['Color_map'] = map(lambda x:color_map[x.split('/')[0].split(' ')[0]],data_test['Color'])

In [15]:
breed_map = dict((w,i+1) for i,w in enumerate(data['Breed'].unique().tolist()))
breed_map['Other'] = 1382

In [16]:
def convert(x):
    try:
        return breed_map[x]
    except:
        return breed_map['Other']

In [17]:
data['Breed_map'] = map(convert,data['Breed'])
data_test['Breed_map'] = map(convert,data_test['Breed'])

In [18]:
week = dict((w,i+1) for i,w in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']))
data['DayOfWeek'] = map(lambda x: week[calendar.day_name[datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S').weekday()]] \
                       ,data['DateTime'])
data_test['DayOfWeek'] = map(lambda x: week[calendar.day_name[datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S').weekday()]] \
                       ,data_test['DateTime'])

In [19]:
##### There is only one 'Dog' with SexuponOutcome == NaN, So we input it as 'Unknown' #####
data.loc[pd.isnull(data['SexuponOutcome']),'SexuponOutcome'] = 'Unknown'

In [20]:
gender = dict((w,i+1) for i,w in enumerate(data['SexuponOutcome'].unique().tolist()))
data['Gender_map'] = map(lambda x:int(gender[x]),data['SexuponOutcome'])
data_test['Gender_map'] = map(lambda x:int(gender[x]),data_test['SexuponOutcome'])

In [21]:
animal = dict((w,i+1) for i,w in enumerate(data['AnimalType'].unique().tolist()))
data['Animal'] = map(lambda x:animal[x],data['AnimalType'])
data_test['Animal'] = map(lambda x:animal[x],data_test['AnimalType'])

In [22]:
outcome = {'Adoption':0,'Died':1,'Euthanasia':2,'Return_to_owner':3,'Transfer':4}
preds = np.asarray(map(lambda x:outcome[x],data['OutcomeType']))

In [23]:
print outcome

{'Transfer': 4, 'Adoption': 0, 'Return_to_owner': 3, 'Died': 1, 'Euthanasia': 2}


In [24]:
data.drop(['Name','DateTime','OutcomeType','OutcomeSubtype','AnimalType',\
           'SexuponOutcome','AgeuponOutcome','Breed','Color'],axis=1,inplace=True)

In [25]:
data_test.drop(['Name','DateTime','AnimalType',\
           'SexuponOutcome','AgeuponOutcome','Breed','Color'],axis=1,inplace=True)

In [26]:
X = data.values
X_test = data_test.values
print X.shape
print X_test.shape

(26711, 6)
(11456, 6)


In [27]:
X_train = X[0:18697,:]
X_cv = X[18697:,:]
print X_train.shape
print X_cv.shape

(18697, 6)
(8014, 6)


In [28]:
nb_train_examples = preds.shape[0]
nb_outcomes = 5
y = np.zeros((nb_train_examples,nb_outcomes))
y[np.arange(nb_train_examples),preds] = 1
print y.shape

(26711, 5)


In [29]:
print y[0:5,:]

[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]


In [30]:
y_train = y[0:18697,:]
y_cv = y[18697:,:]
print y_train.shape
print y_cv.shape

(18697, 5)
(8014, 5)


In [31]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
hist = model.fit(X_train,y_train)

In [32]:
model.score(X_cv,y_cv)

0.53019715497878717

In [50]:
yPreds = np.asarray(model.predict(X_test),dtype='int32')

In [51]:
print yPreds.shape

(11456, 5)


In [49]:
import csv
fname = '/home/najeeb/Desktop/Dataset/Shelter Animals/submission_forest.csv'
try:
    os.remove(fname)
except:
    pass
with open(fname,'wb') as fp:
    p = csv.writer(fp)
    p.writerow(['ID','Adoption','Died','Euthanasia','Return_to_owner','Transfer'])
    for i in data_test.index.tolist():
        p.writerow([int(i)]+list(yPreds[int(i)-1]))